In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, AveragePooling2D, GlobalAveragePooling2D
from keras import regularizers
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import pickle
from sklearn.metrics import confusion_matrix
from PyFunctions import Functions as func
from PyFunctions import Viz
from keras.applications.vgg16 import VGG16
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model

Using plaidml.keras.backend backend.


In [2]:
x_train, x_test, y_train, y_test = func.get_emotion_splits(dim = (224,224), model_type = 'vgg16', max_values = 4000) 
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

TRAIN Value Count
2    4000
1    4000
0    3995
dtype: int64
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
TEST Value Count
1    1774
2    1233
0     958
dtype: int64
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
(11995, 224, 224, 3) (3965, 224, 224, 3)
(11995, 3) (3965, 3)


In [3]:
def get_vgg16(dim):
    model = Sequential()
    baseModel = VGG16(weights="imagenet", include_top=False,
        input_tensor=Input(shape=dim), classes = 3)
    
    # loop over all layers in the base model and freeze them so they will
    # *not* be updated during the first training process
    
        
    model.add(baseModel)
    model.add(GlobalAveragePooling2D())
#     model.add(Flatten(name="flatten"))
    model.add(Dense(1024, activation="relu"))
    model.add(Dense(512, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation="softmax", name = 'Output'))
    
    for layer in baseModel.layers:
        layer.trainable = False
        
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [4]:
early_stopping = EarlyStopping(monitor='val_loss', verbose = 1, patience=8, min_delta = .00075)
model_checkpoint = ModelCheckpoint(f'ModelWeights/VGG16_Emotions.h5', verbose = 1, save_best_only=True,
                                  monitor = 'val_loss')
lr_plat = ReduceLROnPlateau(patience = 5, mode = 'min')
epochs = 2000
batch_size = 32
    

dim = (x_train.shape[1], x_train.shape[2], x_train.shape[3])
vgg16 = get_vgg16(dim =dim)
# vgg16 = load_model('ModelWeights/VGG16_Emotions.h5')
    
augmentation =ImageDataGenerator(rotation_range = 20, width_shift_range = .2, height_shift_range = .2, 
                                                       horizontal_flip = True, shear_range = .15, 
                                 fill_mode = 'nearest', zoom_range = .15)
# augmentation.fit(x_train)
vgg16_history = vgg16.fit_generator(augmentation.flow(x_train, y_train, batch_size = batch_size),
            epochs = epochs, 
     callbacks = [early_stopping, model_checkpoint, lr_plat], validation_data = (x_test, y_test), verbose= 1)

INFO:plaidml:Opening device "opencl_amd_ellesmere.0"


Epoch 1/2000
375/375 [==============================] - 391s 1s/step - loss: 1.1506 - acc: 0.4280 - val_loss: 1.1056 - val_acc: 0.3932

Epoch 00001: val_loss improved from inf to 1.10560, saving model to ModelWeights/VGG16_Emotions.h5
Epoch 2/2000
375/375 [==============================] - 370s 985ms/step - loss: 1.0145 - acc: 0.4894 - val_loss: 0.9870 - val_acc: 0.5246

Epoch 00002: val_loss improved from 1.10560 to 0.98702, saving model to ModelWeights/VGG16_Emotions.h5
Epoch 3/2000
375/375 [==============================] - 369s 985ms/step - loss: 0.9818 - acc: 0.5149 - val_loss: 1.0026 - val_acc: 0.4842

Epoch 00003: val_loss did not improve from 0.98702
Epoch 4/2000
375/375 [==============================] - 369s 984ms/step - loss: 0.9587 - acc: 0.5371 - val_loss: 0.9376 - val_acc: 0.5526

Epoch 00004: val_loss improved from 0.98702 to 0.93761, saving model to ModelWeights/VGG16_Emotions.h5
Epoch 5/2000
375/375 [==============================] - 369s 985ms/step - loss: 0.9464 - ac

KeyboardInterrupt: 

In [ ]:
%%javascript
IPython.notebook.save_notebook()

In [ ]:
os.system(r'%windir%\system32\rundll32.exe powrprof.dll,SetSuspendState Hibernate')